## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login
```

or

```bash
az login --use-device-code
```

# 🏦 Azure AI Agent with Function Tools - Banking Operations 💼

This notebook demonstrates function tool integration with `AzureAIProjectAgentProvider` for use cases, including account management, transaction history, and loan calculations.

## Features Covered:
- Defining function tools for banking operations
- Agent-level tool configuration (tools available for all queries)
- Run-method tool configuration (tools for specific queries)
- Mixed tool usage patterns
- Multiple banking function coordination

### ⚠️ Important Financial Disclaimer ⚠️
> **This notebook demonstrates simulated banking operations for educational purposes. Always use official banking channels for real financial transactions.**

## Prerequisites

Before running this notebook, ensure you have:
- Azure CLI installed and authenticated (`az login --use-device-code`)
- Access to an Microsoft Foundry project with deployed models
- Environment variables set up in `.env` file:
  - `AI_FOUNDRY_PROJECT_ENDPOINT`
  - `AZURE_AI_MODEL_DEPLOYMENT_NAME`

## Import Libraries

Import the required libraries using the `AzureAIAgentsProvider` API:

In [ ]:
import os
from pathlib import Path
from datetime import datetime, timezone
from random import randint, uniform
from typing import Annotated

from agent_framework.azure import AzureAIAgentsProvider
from azure.identity.aio import AzureCliCredential
from pydantic import Field
from dotenv import load_dotenv

# Load environment variables
notebook_path = Path().absolute()
load_dotenv('../../.env')

# Verify environment setup
endpoint = os.getenv('AI_FOUNDRY_PROJECT_ENDPOINT')
model = os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME')

print("🔧 Environment Configuration:")
print(f"✅ Project Endpoint: {endpoint[:50]}..." if endpoint else "❌ AI_FOUNDRY_PROJECT_ENDPOINT not set")
print(f"✅ Model Deployment: {model}" if model else "❌ AZURE_AI_MODEL_DEPLOYMENT_NAME not set")

## Define Function Tools 🏦

Let's define banking-related functions that our agent can use for account operations:

In [ ]:
def get_account_balance(
    account_id: Annotated[str, Field(description="The customer account ID to check balance for.")],
) -> str:
    """Get the current balance for a customer account."""
    # Simulated account balances
    balance = round(uniform(1000, 50000), 2)
    return f"Account {account_id}: Current balance is ${balance:,.2f} as of {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M UTC')}"


def get_transaction_history(
    account_id: Annotated[str, Field(description="The account ID to get transaction history for.")],
    num_transactions: Annotated[int, Field(description="Number of recent transactions to retrieve.")] = 5,
) -> str:
    """Get recent transaction history for an account."""
    transactions = []
    categories = ["Deposit", "Withdrawal", "Transfer", "Payment", "Interest"]
    
    for i in range(num_transactions):
        amount = round(uniform(10, 2000), 2)
        category = categories[randint(0, len(categories)-1)]
        sign = "+" if category in ["Deposit", "Interest"] else "-"
        transactions.append(f"  {i+1}. {category}: {sign}${amount:,.2f}")
    
    return f"Recent transactions for account {account_id}:\n" + "\n".join(transactions)


def calculate_loan_payment(
    principal: Annotated[float, Field(description="The loan principal amount in dollars.")],
    annual_rate: Annotated[float, Field(description="The annual interest rate as a percentage (e.g., 6.5 for 6.5%).")],
    term_months: Annotated[int, Field(description="The loan term in months.")],
) -> str:
    """Calculate the monthly payment for a loan."""
    monthly_rate = annual_rate / 100 / 12
    if monthly_rate == 0:
        monthly_payment = principal / term_months
    else:
        monthly_payment = principal * (monthly_rate * (1 + monthly_rate)**term_months) / ((1 + monthly_rate)**term_months - 1)
    
    total_payment = monthly_payment * term_months
    total_interest = total_payment - principal
    
    return f"""Loan Calculation:
  Principal: ${principal:,.2f}
  Annual Rate: {annual_rate}%
  Term: {term_months} months
  Monthly Payment: ${monthly_payment:,.2f}
  Total Interest: ${total_interest:,.2f}
  Total Payment: ${total_payment:,.2f}"""

## Pattern 1: Tools Defined on Agent Level 🔧

In this pattern, tools are provided when creating the agent. The agent can use these banking tools for any query during its lifetime:

In [ ]:
async def tools_on_agent_level() -> None:
    """Example showing tools defined when creating the agent."""
    print("=== 🏦 Pattern 1: Tools Defined on Agent Level ===")

    async with (
        AzureCliCredential() as credential,
        AzureAIAgentsProvider(credential=credential) as provider,
    ):
        # Tools are provided when creating the agent
        agent = await provider.create_agent(
            name="banking-assistant-agent",
            instructions="You are a helpful Banking Assistant that can check account balances, show transaction history, and calculate loan payments.",
            tools=[get_account_balance, get_transaction_history, calculate_loan_payment],
        )
        
        # First query - agent uses account balance tool
        query1 = "What's the balance for account ACC-12345?"
        print(f"\n🤔 Customer: {query1}")
        result1 = await agent.run(query1)
        print(f"🏦 Assistant: {result1}\n")

        # Second query - agent uses transaction history tool
        query2 = "Show me the last 3 transactions for account ACC-12345"
        print(f"🤔 Customer: {query2}")
        result2 = await agent.run(query2)
        print(f"🏦 Assistant: {result2}\n")

        # Third query - agent uses loan calculation tool
        query3 = "Calculate the monthly payment for a $250,000 mortgage at 6.5% for 30 years"
        print(f"🤔 Customer: {query3}")
        result3 = await agent.run(query3)
        print(f"🏦 Assistant: {result3}\n")

# Run the agent-level tools example
await tools_on_agent_level()

## Pattern 2: Tools Passed to Run Method 🎯

In this pattern, tools are passed to the `run` method, allowing for different banking tools per query:

In [ ]:
async def tools_on_run_level() -> None:
    """Example showing tools passed to the run method for specific queries."""
    print("=== 🎯 Pattern 2: Tools Passed to Run Method ===")

    async with (
        AzureCliCredential() as credential,
        AzureAIAgentsProvider(credential=credential) as provider,
    ):
        # Agent created without tools
        agent = await provider.create_agent(
            name="flexible-banking-agent",
            instructions="You are a helpful Banking Assistant. Use the tools provided to help customers with their banking needs.",
        )
        
        # First query with account balance tool only
        query1 = "What's my balance for account ACC-98765?"
        print(f"\n🤔 Customer: {query1}")
        result1 = await agent.run(query1, tools=[get_account_balance])
        print(f"🏦 Assistant: {result1}\n")

        # Second query with loan calculation tool only
        query2 = "Calculate payments for a $50,000 auto loan at 7% for 5 years"
        print(f"🤔 Customer: {query2}")
        result2 = await agent.run(query2, tools=[calculate_loan_payment])
        print(f"🏦 Assistant: {result2}\n")

        # Third query with multiple tools
        query3 = "Check balance for ACC-98765 and calculate a personal loan of $10,000 at 10% for 3 years"
        print(f"🤔 Customer: {query3}")
        result3 = await agent.run(query3, tools=[get_account_balance, calculate_loan_payment])
        print(f"🏦 Assistant: {result3}\n")

# Run the run-level tools example
await tools_on_run_level()

## Pattern 3: Mixed Tools (Agent + Run Method) 🔄

This pattern combines agent-level tools with additional run-method tools for comprehensive banking services:

In [ ]:
async def mixed_tools_example() -> None:
    """Example showing both agent-level tools and run-method tools together."""
    print("=== 🔄 Pattern 3: Mixed Tools (Agent + Run Method) ===")

    async with (
        AzureCliCredential() as credential,
        AzureAIAgentsProvider(credential=credential) as provider,
    ):
        # Agent created with base tools (always available)
        agent = await provider.create_agent(
            name="comprehensive-banking-agent",
            instructions="You are a comprehensive Banking Assistant. Help customers with account inquiries and loan calculations.",
            tools=[get_account_balance],  # Base tool available for all queries
        )
        
        # Query using base tool + additional run-method tools
        query = "Check balance for ACC-54321, show last 4 transactions, and calculate a $100,000 home equity loan at 8% for 15 years"
        print(f"\n🤔 Customer: {query}")

        # Agent has access to get_account_balance (from creation) + additional tools
        result = await agent.run(
            query,
            tools=[get_transaction_history, calculate_loan_payment],  # Additional tools for this query
        )
        print(f"🏦 Assistant: {result}\n")

# Run the mixed tools example
await mixed_tools_example()

## Advanced Function Tools 📊

Let's create more sophisticated banking tools for investment and credit operations:

In [ ]:
def get_credit_score(
    customer_id: Annotated[str, Field(description="The customer ID to check credit score for.")],
) -> str:
    """Get the credit score for a customer."""
    score = randint(580, 850)
    rating = "Excellent" if score >= 750 else "Good" if score >= 700 else "Fair" if score >= 650 else "Poor"
    return f"Customer {customer_id}: Credit Score is {score} ({rating})"


def get_investment_portfolio(
    account_id: Annotated[str, Field(description="The investment account ID.")],
) -> str:
    """Get investment portfolio summary."""
    stocks = round(uniform(10000, 50000), 2)
    bonds = round(uniform(5000, 25000), 2)
    cash = round(uniform(1000, 10000), 2)
    total = stocks + bonds + cash
    
    return f"""Investment Portfolio for {account_id}:
  Stocks: ${stocks:,.2f} ({stocks/total*100:.1f}%)
  Bonds: ${bonds:,.2f} ({bonds/total*100:.1f}%)
  Cash: ${cash:,.2f} ({cash/total*100:.1f}%)
  Total Value: ${total:,.2f}
  Last Updated: {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M UTC')}"""


def calculate_compound_interest(
    principal: Annotated[float, Field(description="Initial investment amount.")],
    annual_rate: Annotated[float, Field(description="Annual interest rate as percentage.")],
    years: Annotated[int, Field(description="Number of years.")],
    compounds_per_year: Annotated[int, Field(description="Number of times interest compounds per year (12 for monthly, 4 for quarterly).")] = 12,
) -> str:
    """Calculate compound interest growth."""
    final_amount = principal * (1 + annual_rate/100/compounds_per_year) ** (compounds_per_year * years)
    total_interest = final_amount - principal
    
    return f"""Compound Interest Calculation:
  Initial Investment: ${principal:,.2f}
  Annual Rate: {annual_rate}%
  Term: {years} years
  Compounding: {compounds_per_year}x per year
  Final Value: ${final_amount:,.2f}
  Total Interest Earned: ${total_interest:,.2f}"""

## Comprehensive Financial Advisor Example 💼

Let's test a comprehensive Financial Advisor with all banking tools:

In [ ]:
async def comprehensive_financial_advisor():
    """Comprehensive Financial Advisor with all banking tools."""
    print("=== 💼 Comprehensive Financial Advisor ===")

    async with (
        AzureCliCredential() as credential,
        AzureAIAgentsProvider(credential=credential) as provider,
    ):
        # Create a comprehensive financial advisor
        agent = await provider.create_agent(
            name="financial-advisor-agent",
            instructions="""You are a comprehensive Financial Advisor that can help customers with:
            - Checking account balances and transaction history
            - Credit score inquiries
            - Investment portfolio reviews
            - Loan calculations and planning
            - Compound interest projections
            Always provide professional, accurate information and remind customers to consult professionals for major financial decisions.""",
            tools=[
                get_account_balance,
                get_transaction_history,
                calculate_loan_payment,
                get_credit_score,
                get_investment_portfolio,
                calculate_compound_interest
            ],
        )
        
        queries = [
            "Check my credit score for customer ID CUST-001 and show my investment portfolio for INV-001",
            "If I invest $25,000 at 7% annual interest compounded monthly for 20 years, how much will I have?",
            "I want to take a $200,000 mortgage at 6.25% for 30 years. What's my monthly payment and can you also check my account ACC-001 balance?"
        ]
        
        for i, query in enumerate(queries, 1):
            print(f"\n{'='*60}")
            print(f"--- 💼 Financial Consultation {i} ---")
            print(f"🤔 Customer: {query}")
            print("-"*40)
            result = await agent.run(query)
            print(f"🏦 Advisor: {result}")
        
        print("\n⚠️ Disclaimer: All financial information is simulated for demonstration purposes.")

# Run comprehensive financial advisor
await comprehensive_financial_advisor()

## Full Banking Consultation Example 🏦

A complete banking consultation scenario:

In [ ]:
async def banking_consultation_scenario():
    """Simulate a complete banking consultation scenario."""
    print("=== 🏦 Complete Banking Consultation Scenario ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentsProvider(credential=credential) as provider,
    ):
        agent = await provider.create_agent(
            name="banking-consultant",
            instructions="""You are a senior Banking Consultant. Guide customers through their financial review:
            1. First check their current account status
            2. Review their credit standing
            3. Analyze investment portfolio
            4. Calculate potential loan options
            Be thorough, professional, and educational.""",
            tools=[
                get_account_balance,
                get_transaction_history,
                get_credit_score,
                get_investment_portfolio,
                calculate_loan_payment,
                calculate_compound_interest
            ],
        )
        
        # Multi-step consultation
        print("\n📋 Starting comprehensive financial review for customer CUST-2024...")
        
        review_request = """Please conduct a complete financial review for customer CUST-2024:
        1. Check checking account ACC-2024-CHK balance
        2. Show last 5 transactions
        3. Check credit score
        4. Review investment portfolio INV-2024
        5. Calculate what a $150,000 home loan at 6.75% for 30 years would cost
        6. Project growth of $10,000 invested at 8% for 10 years"""
        
        print(f"🤔 Review Request: {review_request}\n")
        print("-"*60)
        result = await agent.run(review_request)
        print(f"📊 Comprehensive Review:\n{result}")
        
        print("\n" + "="*60)
        print("✅ Financial consultation complete!")
        print("⚠️ Note: All data is simulated for demonstration purposes.")

# Run the banking consultation
await banking_consultation_scenario()

## Key Takeaways 📚

### API Pattern

```python
async with (
    AzureCliCredential() as credential,
    AzureAIAgentsProvider(credential=credential) as provider,
):
    agent = await provider.create_agent(
        name="my-agent",  # Use hyphens, not underscores
        instructions="...",
        tools=[tool1, tool2],
    )
```

### Tool Configuration Patterns

1. **Agent-Level Tools**: Defined at agent creation, available for all queries
2. **Run-Level Tools**: Passed to specific `run()` calls
3. **Mixed Tools**: Combine both patterns for flexible tool access

### Function Tools Demonstrated

| Tool | Purpose |
|------|---------|
| `get_account_balance` | Check account balances |
| `get_transaction_history` | View recent transactions |
| `calculate_loan_payment` | Calculate loan payments |
| `get_credit_score` | Check credit scores |
| `get_investment_portfolio` | Review investment holdings |
| `calculate_compound_interest` | Project investment growth |

### Best Practices

1. **Tool Typing**: Use `Annotated` and `Field` for clear parameter descriptions
2. **Docstrings**: Include helpful docstrings for tool functions
3. **Error Handling**: Handle missing or invalid inputs gracefully
4. **Agent Names**: Use hyphens (not underscores) in agent names
5. **Disclaimers**: Always include appropriate financial disclaimers

### Use Cases

- **Account Management**: Balance checks, transaction history
- **Loan Services**: Payment calculations, rate comparisons
- **Investment Services**: Portfolio review, growth projections
- **Credit Services**: Credit score checks, eligibility assessments

⚠️ **Important**: All financial functions in this notebook use simulated data for demonstration. Real banking applications require secure connections to actual banking systems.